<a href="https://colab.research.google.com/github/giuliamf/tag_projeto2/blob/main/tag_projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pandas networkx matplotlib ace_tools

In [8]:
import sys

In [5]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/tag'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Função auxiliar para visualizar o grafo durante o desenvolvimento

In [6]:
from visualizar_grafo import visualizar_grafo_bipartido

ModuleNotFoundError: No module named 'visualizar_grafo'

Trata os dados do arquivo dados.txt

In [ ]:
import pandas as pd
import re

In [ ]:
with open('dados.txt', 'r', encoding='utf-8') as file:
  linhas = file.readlines()

linhas = [linha.strip() for linha in linhas if linha.strip()]

split_id = next(i for i, l in enumerate(linhas) if l.startswith('//alunos'))

projeto_linhas = linhas[2:split_id]
estudante_linhas = linhas[split_id+2:]

Trata os dados dos projetos em um dataframe

In [ ]:
projetos = []
for linha in projeto_linhas:
  match = re.match(r'\((P\d+),\s*(\d+),\s*(\d+)\)', linha)
  if match:
    codigo, vagas, nota_min = match.groups()
    projetos.append({'projeto': codigo, 'vagas': int(vagas), 'nota_min': int(nota_min)})

df_projetos = pd.DataFrame(projetos)

In [ ]:
df_projetos.head()

Trata os dados dos estudantes em um dataframe

In [ ]:
alunos = []
for linha in estudante_linhas:
  match = re.match(r'\((A\d+)\):\((P\d+(?:,\s*P\d+){0,2})\)\s*\((\d+)\)', linha)
  if match:
    codigo, preferencias, nota = match.groups()
    prefs = [p.strip() for p in preferencias.split(',')]
    alunos.append({
        'aluno': codigo,
        'pref_1': prefs[0] if len(prefs) > 0 else None,
        'pref_2': prefs[1] if len(prefs) > 1 else None,
        'pref_3': prefs[2] if len(prefs) > 2 else None,
        'nota': int(nota)
        })

df_alunos = pd.DataFrame(alunos)

In [ ]:
df_alunos.head()

Começando a criar um grafo bipartido com os dados tratados acima

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

cria o grafo

In [ ]:
grafo = nx.Graph()

- adiciona os alunos - bipartido 1
- faz o dicionario de requisitos para o projeto


In [ ]:
grafo.add_nodes_from(df_alunos['aluno'], bipartite=0)

requisitos = df_projetos.set_index('projeto').to_dict('index')

- adiciona as vagas do projeto - bipartido 2

In [ ]:
for _, row in df_projetos.iterrows():
  for i in range(row['vagas']):
    vaga = f'{row["projeto"]}-{i+1}'
    grafo.add_node(vaga, bipartite=1)

- cria as arestas: aluno -> vaga do projeto se a nota for maior ou igual o requisito

In [ ]:
for _, aluno in df_alunos.iterrows():
  aluno_id = aluno['aluno']
  nota = aluno['nota']
  for pref in ['pref_1', 'pref_2', 'pref_3']:
    projeto = aluno[pref]
    if projeto in requisitos and nota >= requisitos[projeto]['nota_min']:
      for i in range(requisitos[projeto]['vagas']):
        vaga = f'{projeto}-{i+1}'
        grafo.add_edge(aluno_id, vaga)

informações do grafo até aqui

In [ ]:
n_alunos = len([n for n, d in grafo.nodes(data=True) if d.get("bipartite") == 0])
n_vagas = len([n for n, d in grafo.nodes(data=True) if d.get("bipartite") == 1])
n_arestas = grafo.number_of_edges()

df_stats = pd.DataFrame({
    "Tipo": ["Alunos", "Vagas", "Arestas"],
    "Quantidade": [n_alunos, n_vagas, n_arestas]
})

df_stats

In [ ]:
visualizar_grafo_bipartido(grafo, max_nos=20)

Emparelhamento estável - aluno: projeto

In [ ]:
pref_alunos = []

for _, aluno in df_alunos.iterrows():
  aluno_id = aluno['aluno']
  aluno_nota = aluno['nota']

  for pref in ['pref_1', 'pref_2', 'pref_3']:
    projeto = aluno[pref]
    if projeto in requisitos:
      req = requisitos[projeto]
      if aluno_nota >= req['nota_min']:
        for i in range(req['vagas']):
          pref_alunos.append({
              'aluno': aluno_id,
              'projeto': projeto,
              'vaga': f'{projeto}-{i+1}',
              'ordem': int(pref[-1])
          })

df_pref_alunos = pd.DataFrame(pref_alunos)

In [ ]:
df_pref_alunos.head()

projeto: notas

In [ ]:
pref_vagas = []

for _, projeto in df_projetos.iterrows():
    for i in range(projeto['vagas']):
        vaga_id = f'{projeto["projeto"]}-{i+1}'

        alunos_elegiveis = df_alunos[
            df_alunos[['pref_1', 'pref_2', 'pref_3']].apply(lambda row: projeto['projeto'] in row.values, axis=1) &
            (df_alunos['nota'] >= projeto['nota_min'])
        ].sort_values(by='nota', ascending=False)['aluno'].tolist()

        for i, aluno in enumerate(alunos_elegiveis, 1):
            pref_vagas.append({
                'vaga': vaga_id,
                'aluno': aluno,
                'ordem_preferencia': i  # 1 para o aluno com maior nota, 2 para o segundo, etc.
            })

df_pref_vagas = pd.DataFrame(pref_vagas)

In [ ]:
df_pref_vagas.head()